# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [18]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("tic-tac-toe.csv")

print(df.head())
print(df.info())

x_encoded = pd.get_dummies(df.drop("class", axis = 1))
y = df["class"].astype(int)

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_encoded)

  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [19]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Input(shape=(x_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

model.save("tic_tac_toe_model.h5")


Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5555 - loss: 0.7057 - val_accuracy: 0.6494 - val_loss: 0.6142
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - accuracy: 0.7047 - loss: 0.5664 - val_accuracy: 0.6818 - val_loss: 0.5865
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - accuracy: 0.7543 - loss: 0.4998 - val_accuracy: 0.6948 - val_loss: 0.5539
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - accuracy: 0.7810 - loss: 0.4543 - val_accuracy: 0.7013 - val_loss: 0.5339
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - accuracy: 0.7672 - loss: 0.4395 - val_accuracy: 0.7727 - val_loss: 0.5036
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - accuracy: 0.8260 - loss: 0.4014 - val_accuracy: 0.7468 - val_loss: 0.4776
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - accuracy: 0.8594 - loss: 0.3646 - val_accuracy: 0.7922 - val_loss: 0.4459
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - accuracy: 0.8839 - loss: 0.3134 - val_accuracy: 0.82

Test Loss: 0.08426517993211746, Test Accuracy: 0.96875


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [20]:
import tensorflow as tf
import numpy as np
model = tf.keras.models.load_model("tic_tac_toe_model.h5")
sample_data = x_test[:5]
sample_labels = y_test[:5]
predictions = model.predict(sample_data)
sample_labels = np.array(sample_labels)
for i in range(len(sample_data)):
    predicted = 1 if predictions[i] > 0.5 else 0
    actual = sample_labels[i]
    print(f"Sample {i+1}: Predicted = {predicted}, Actual = {actual}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Sample 1: Predicted = 1, Actual = 0
Sample 2: Predicted = 1, Actual = 1
Sample 3: Predicted = 1, Actual = 1
Sample 4: Predicted = 0, Actual = 0
Sample 5: Predicted = 0, Actual = 0


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [21]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dense(32, activation='relu'))  
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

loss, accuracy = model.evaluate(x_test, y_test)
print(f"Improved Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

Epoch 1/50


/opt/anaconda3/envs/ironhack/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5748 - loss: 0.6834 - val_accuracy: 0.6623 - val_loss: 0.6326
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - accuracy: 0.6784 - loss: 0.5871 - val_accuracy: 0.7338 - val_loss: 0.5868
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step - accuracy: 0.7501 - loss: 0.5440 - val_accuracy: 0.7662 - val_loss: 0.5547
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - accuracy: 0.7706 - loss: 0.4918 - val_accuracy: 0.7468 - val_loss: 0.5270
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - accuracy: 0.7688 - loss: 0.4805 - val_accuracy: 0.7597 - val_loss: 0.5057
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 934us/step - accuracy: 0.8159 - loss: 0.4185 - val_accuracy: 0.7792 - val_loss: 0.4810
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - accuracy: 0.8114 - loss: 0.4124 - val_accuracy: 0.7857 - val_loss: 0.4536
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.8479 - loss: 0.3764 - val_accuracy: 0.7792 - val_lo

**Which approach(es) did you find helpful to improve your model performance?**

In [22]:
# adding an additional layer helped improve the model's performance
# using Adam optimizer with a learning rate of 0.001 also contributed to better convergence
# these changes led to a more robust model that generalizes better on unseen data